## 5.2 利用nn.Sequence组合子模块

5.1节中构建了一个一元线性回归模型，那是一个极其简单的模块，只需要直接调用nn.Linear类即可。

但是实际上我们往往需要去构建更复杂一点的模型，这就需要我们调用不同的子模块然后将它们拼接起来组成大模块。比如先经过一个线性函数，再经过tanh激活函数，再经过一个线性函数，这是三个步骤的组合,前者的输出事后者的输入。

如何组合子模块成一个完整的模型，可以用nn.Sequence。

### 5.2.1 直接创建

In [1]:
import torch.nn as nn

seq_model = nn.Sequential(
    nn.Linear(1,13),
    nn.Tanh(),
    nn.Linear(13,1)
)

seq_model

Sequential(
  (0): Linear(in_features=1, out_features=13, bias=True)
  (1): Tanh()
  (2): Linear(in_features=13, out_features=1, bias=True)
)

- 代码的解释：
这样，我们创建了一个模型，里面有三个子模块，分别是Linean,tanh,Linear,前者的输出是后者的出入，因此注意维度的对应，第一个Linear是输入1输出13，tanh不改变维度，第二个Linear的输入需要与第一个Linear的输出保持一致。

- 打印信息的解释
括号中数字的是每个子模块的名字，按照其再过程中的顺序默认以标号来取名。标号后面是每个子模块，以及其输入输出的维度。

### 5.2.2 带名字创建
如果不想用默认的标号作为子模块的名字，可以用OrderedDict的字典模式创建，并取上自己的想要的名字，以方便后续调用子模块。

In [4]:
from collections import OrderedDict

seq_model = nn.Sequential(OrderedDict([
    ('hidden_linear',nn.Linear(1,13)),
    ('hidden_activation', nn.Tanh()),
    ('output_linear', nn.Linear(13,1))
    
]))

seq_model

Sequential(
  (hidden_linear): Linear(in_features=1, out_features=13, bias=True)
  (hidden_activation): Tanh()
  (output_linear): Linear(in_features=13, out_features=1, bias=True)
)

### 5.2.3 获取参数
#### （1）model.parameters

model.parameters是一个generator,需要循环打印,其中依次包含了从前至后的计算中涉及的参数，如下分别是hidden_linear中的w,b和output_linear中的w,b的shape

In [8]:
[param.shape for param in seq_model.parameters()]

[torch.Size([13, 1]), torch.Size([13]), torch.Size([1, 13]), torch.Size([1])]

#### （2）model.named_parameters
直接用model.parameters打印出来的信息要按照顺序去数着对应，很不方便，用named_parameters可以打印出参数对应子模块的名字，很是便利

In [9]:
for name, param in seq_model.named_parameters():
    print(name, param.shape)

hidden_linear.weight torch.Size([13, 1])
hidden_linear.bias torch.Size([13])
output_linear.weight torch.Size([1, 13])
output_linear.bias torch.Size([1])


#### （3）model.submodule.weight
除了以上方法，可以直接调用子模块的参数，如：

In [10]:
seq_model.output_linear.bias

Parameter containing:
tensor([0.0821], requires_grad=True)

同理，也可以同过这种方式获取参数对应的梯度

In [13]:
print(seq_model.output_linear.weight.grad)

None
